# Explore clustering algorithm confidence discounting
The proposed clustering algorithm confidence discounting algorithm (https://github.com/e-mission/e-mission-docs/issues/663#issuecomment-898994131) has several parameters that may be tuned. This is to explore what they should be set to and whether the algorithm works well overall.

For now, I will not do the full testing, which would consist of a train/test split, running the clustering on only the training data, then comparing the calculated p-values to how well each cluster actually does at predicting test trips.

First, let's get some users. We select only users who have at least 30 confirmed trips.

In [ ]:
# Copypasted from Explore stage before vs after; TODO refactor into a module
import emission.storage.timeseries.abstract_timeseries as esta

EXCLUDE_UUIDS = [UUID(s) for s in input("Enter UUIDs to exclude, separated by spaces: ").split(" ") if len(s) > 0]
REQUIRED_TRIPS_TOTAL = 30

def filter_update(new, old, reason):
    print(f"Excluded {len(old)-len(new)} users, left with {len(new)}: {reason}")

all_users = esta.TimeSeries.get_uuid_list()
confirmed_trip_df_map = {}
print(f"Working with {len(all_users)} initial users")

filter0_users = [u for u in all_users if u not in EXCLUDE_UUIDS]  # Users that we don't explicitly exclude
filter_update(filter0_users, all_users, "presence on exclusion list")

filter1_users = []  # Users with enough total trips
for u in filter0_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")
    confirmed_trip_df_map[u] = ct_df
    if ct_df.shape[0] >= REQUIRED_TRIPS_TOTAL: filter1_users.append(u)
filter_update(filter1_users, filter0_users, "not enough total trips")

filtered_users = filter1_users

Now let's get all the cleaned trips for those users and figure out what the naïve predictions would be. Note that this requires the model files to be copied into the working directory.

In [ ]:
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.analysis.classification.inference.labels.inferrers as eacili
import arrow
import emission.storage.timeseries.timequery as estt
import uuid
import emission.analysis.modelling.tour_model.data_preprocessing as preprocess
import emission.analysis.modelling.tour_model_first_only.load_predict as lp

EMPTY_INFERENCE = {"labels": {}, "p": 0.0}
def mli(labels):  # Most likely inference
    return max([e for e in labels], key = lambda e : e["p"]) if len(labels) > 0 else EMPTY_INFERENCE

cleaned_trips = []
findings = []
naive_ps = {0.0, 1.0}
naive_counts = {}
for u in filtered_users:
    tq = estt.TimeQuery("data.end_ts", arrow.get("2010-01-01").timestamp, arrow.now().timestamp)
    cleaned_trips += esda.get_entries(esda.CLEANED_TRIP_KEY, u, time_query=tq)
for trip in cleaned_trips:
    finding = {}
    finding["trip"] = trip
    finding["naive_prediction"] = eacili.predict_two_stage_bin_cluster(trip)
    finding["naive_mli_p"] = mli(finding["naive_prediction"])["p"]
    naive_ps.add(finding["naive_mli_p"])
    if finding["naive_mli_p"] not in naive_counts: naive_counts[finding["naive_mli_p"]] = 0
    naive_counts[finding["naive_mli_p"]] += 1
    findings.append(finding)
    
print(len(cleaned_trips))
print(len(findings))

Now let's compute discounted predictions and test out our graphing functionality.

In [ ]:
# The discount data is time-consuming to calculate, so we cache it as a constant tuple: data dict
discount_cache = {}  # As long as we specify constant tuples as literals, we shouldn't have to deal with floating-point error in matching to the cache

discounted_ps = {0.0, 1.0}
discounted_counts = {}

In [ ]:
def compute_discounting_full(max_confidence=None, first_confidence=None, confidence_multiplier=None):
    global discounted_ps, discounted_counts
    if (max_confidence, first_confidence, confidence_multiplier) in discount_cache:
        discounted_ps, discounted_counts = discount_cache[(max_confidence, first_confidence, confidence_multiplier)]
        return
    
    discounted_ps = {0.0, 1.0}
    discounted_counts = {}
    for finding in findings:
        finding["discounted_prediction"] = eacili.predict_cluster_confidence_discounting(finding["trip"], max_confidence, first_confidence, confidence_multiplier)
        finding["discounted_mli_p"] = mli(finding["discounted_prediction"])["p"]
        discounted_ps.add(finding["discounted_mli_p"])
        if finding["discounted_mli_p"] not in discounted_counts: discounted_counts[finding["discounted_mli_p"]] = 0
        discounted_counts[finding["discounted_mli_p"]] += 1
    discount_cache[(max_confidence, first_confidence, confidence_multiplier)] = (discounted_ps, discounted_counts)
    
import numpy as np
import matplotlib.pyplot as plt
import copy
import warnings

def bar(labels, a, b, title, figsize):
    x = np.arange(len(labels))
    y_a = [a[k] if k in a else 0 for k in labels]
    y_b = [b[k] if k in b else 0 for k in labels]

    fig,ax = plt.subplots(figsize=figsize)
    width = 0.4
    bars_a = ax.bar(x-width/2, y_a, width, label="Naïve")
    bars_b = ax.bar(x+width/2, y_b, width, label="Discounted")

    ax.set_ylabel("Number of trips")
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{n:.2f}" for n in labels])
    ax.legend()
    
    for i,l in enumerate(ax.xaxis.get_ticklabels()):
        if i < 6 or i >= len(labels)-6: l.set_rotation(90)
        elif i % 4 != 0: l.set_visible(False)  # Hide labels that don't fit
    
    plt.show()

def expand_dict(src, dest):
    for k in src: dest += [float(k)]*int(src[k])
    
def box(a, b, title, figsize, first_confidence):
    data = [[] for i in range(4)]
    labels = ["Naïve", "Discounted", "Naïve no 0", "Discounted no 0", "Naïve no 0, 1", "Discounted no 0, B"]
    expand_dict(a, data[0])
    expand_dict(b, data[1])
    a_no_0 = a.copy()
    a_no_0[0] = 0
    b_no_0 = b.copy()
    b_no_0[0] = 0
    expand_dict(a_no_0, data[2])
    expand_dict(b_no_0, data[3])
    
    fig,ax = plt.subplots(figsize=figsize)
    ax.set_title(title)
    ax.set_xticklabels(labels)
    
    warnings.filterwarnings("ignore") # https://github.com/matplotlib/matplotlib/issues/16353
    ax.boxplot(data)
    warnings.resetwarnings()
    
def viz_discounting(max_confidence, first_confidence, confidence_multiplier, bar_figsize=(20,10), box_figsize=(10,5), title_add=""):
    compute_discounting_full(max_confidence, first_confidence, confidence_multiplier)
    print(f"Not shown: {naive_counts[0.0]}, {discounted_counts[0.0]} trips with confidence 0")
    labels = list((naive_ps | discounted_ps) - {0.0})
    labels.sort()
    title = f"A={1-max_confidence:.2f}, B={first_confidence:.2f}, C={confidence_multiplier:.2f}"+title_add
    bar(labels, naive_counts, discounted_counts, title, bar_figsize)
    box(naive_counts, discounted_counts, title, box_figsize, first_confidence)
    
viz_discounting(0.99, 0.75, 0.25)


And now let's use the graphs to test out a bunch of scenarios.

In [ ]:
scenarios = {
    "default": (0.99, 0.75, 0.25),
    "no_a": (1.00, 0.75, 0.25),
    "high_a": (0.95, 0.75, 0.25),
    "higher_a": (0.90, 0.75, 0.25),
    "low_b": (0.99, 0.6, 0.25),
    "lower_b": (0.99, 0.3, 0.25),
    "low_c": (0.99, 0.5, 0.10),
    "low_b_and_c": (0.99, 0.6, 0.10)
}

for scenario in scenarios:
    viz_discounting(*scenarios[scenario], title_add=f" ({scenario})")

A quick check on what keys we have to work with

In [ ]:
discount_cache.keys()

Now let's make some better boxplots

In [ ]:
def expand_cache(scenario, remove_0=False):
    compute_discounting_full(*scenario)
    dest = []
    d = discounted_counts.copy()
    if remove_0: d[0] = 0
    expand_dict(d, dest)
    return dest

def box_sidebyside(scenarios, remove_0=True):
    labels = ["naïve\n(no discounting)"]+list(scenarios.keys())
    for i,label in enumerate(labels):
        if i == 0: continue
        s = scenarios[label]
        labels[i] += f"\n({s[0]:.2f}, {s[1]:.2f}, {s[2]:.2f})"
        naive_counts_no_0 = naive_counts.copy()
        if remove_0: naive_counts_no_0[0] = 0
        naive_no_0 = []
        expand_dict(naive_counts_no_0, naive_no_0)
    data = [naive_no_0]+[expand_cache(s, remove_0=remove_0) for s in scenarios.values()]
    
    fig,ax = plt.subplots(figsize=(15,5))
    ax.set_title(f"Side-by-side scenario comparison, most likely inference, {'NOT ' if not remove_0 else ''}removing 0")
    ax.set_xticklabels(labels)
    ax.set_ylim([-0.05, 1.05])
    
    warnings.filterwarnings("ignore") # https://github.com/matplotlib/matplotlib/issues/16353
    ax.boxplot(data)
    ax.grid(axis="y")
    warnings.resetwarnings()
    
    
box_sidebyside(scenarios)

And let's try some more scenarios altering just B and C:

In [ ]:
scenarios2 = {
    "default": (0.99, 0.75, 0.25),
    "low_b": (0.99, 0.6, 0.25),
    "low_c": (0.99, 0.5, 0.10),
    "low_b_and_c": (0.99, 0.6, 0.10),
    "high_b": (0.99, 0.9, 0.25),
    "high_c": (0.99, 0.75, 0.33),
    "high_b_and_c": (0.99, 0.9, 0.33)
}
box_sidebyside(scenarios2)

In [ ]:
for scenario in scenarios2:
    print(scenario)
    print([f"{eacili.n_to_confidence_coeff(n, *scenarios2[scenario]):.3f}" for n in range(1,11)])
    print()                        

Let's try to get a B and a C by working backwards. We'll say that under relaxed mode, we need
 1. 1 occurrence of a common trip before it shows up as yellow
 2. 4 occurrences of a common trip before the user doesn't need to interact with it at all
 3. If the first occurrence of a common trip has label tuple X, we need 1 occurrence with label tuple Y before we show label tuple Y as yellow
 4. If the first occurrence of a common trip has label tuple X, we need 10 occurrences with label tuple Y before we predict label tuple Y and don't require any user interaction

and under intensive mode we need
 1. 1 occurrence of a common trip before it shows up as yellow
 2. Infinite occurrences of a common trip before the user doesn't need to interact with it at all (in intensive mode, we want to capture things like the user drove alone for years but now they're carpooling)
 3. If the first occurrence of a common trip has label tuple X, we need 2 occurrences with label tuple Y before we show label tuple Y as yellow
 4. If the first occurrence of a common trip has label tuple X, we need infinite occurrences with label tuple Y before we predict label tuple Y and don't require any user interaction

From this we deduce:
 * Relaxed 3 implies that our intensive low confidence threshold should be roughly between 0.00 and 0.50
 * Relaxed 4 implies that our relaxed high confidence threshold should be roughly between 0.90 and 0.91
 * Intensive 3 implies that our intensive low confidence threshold should be roughly between 0.50 and 0.66
 * Intensive 4 implies that our intensive high confidence threshold should be roughly between A and 1

Let us set:
 * Relaxed low confidence threshold: 0.40
 * Relaxed high confidence threshold: 0.90
 * Intensive low confidence threshold: 0.60
 * Intensive high confidence threshold: 0.99 (or whatever A is if we change it)

Now that we have that, we can deduce some more:
 * Relaxed 1 implies that B is greater than 0.40
 * Relaxed 2 implies that the discount for `n=4` should be between 0.90 and 1.00 and the discount for `n=3` should be less than or equal to 0.90
 * Intensive 1 implies that B is greater than 0.60

If we fix `B=0.85`, what do we need C to be to fulfill Relaxed 2 (and the rest of the criteria)?

In [ ]:
def search_for_c(A, B, n_range, search_range):
    print("Searching for C:")
    for C in search_range:
        print(f"({1-A:.2f}, {B:.2f}, {C:.2f})")
        print([f"{eacili.n_to_confidence_coeff(n, 1-A, B, C):.3f}" for n in n_range])
        print()
search_for_c(0.01, 0.85, [3,4], np.arange(0.10, 0.40, 0.05))
search_for_c(0.01, 0.85, [3,4], np.arange(0.12, 0.20, 0.01))

So C can be 0.14 through 0.20. That's on the low side. Lowering B would let us raise C. How much? If we fix `B=0.75`:

In [ ]:
search_for_c(0.01, 0.75, [3,4], np.arange(0.10, 0.40, 0.05))
search_for_c(0.01, 0.75, [3,4], np.arange(0.25, 0.40, 0.01))

So now C can be in the range 0.29 through 0.38. What about `B=0.80`?

In [ ]:
search_for_c(0.01, 0.80, [3,4], np.arange(0.10, 0.40, 0.05))
search_for_c(0.01, 0.80, [3,4], np.arange(0.20, 0.35, 0.01))

Here it's 0.23 through 0.31.

Let us preliminarily propose `A=0.01, B=0.80, C=0.30` and verify that it meets the criteria above.

In [ ]:
print({n: f"{eacili.n_to_confidence_coeff(n, 0.99, 0.80, 0.30):.3f}" for n in range(1,16)})

Everything checks out, except we require 11 instead of 10 for Relaxed 4. We could remedy this either by changing Relaxed 4 to 11 or by changing relaxed high confidence threshold to 0.89. I propose the latter.

### Final proposal
Thus, my final proposal is:

Discounting constants:
 * `A=0.01`, `B=0.80`, `C=0.30`
 
Configuration values:
 * Relaxed mode: low confidence threshold: `0.40`; high confidence threshold: `0.89`
 * Intensive mode: low confidence threshold: `0.60`; high confidence threshold: `0.99`

For completeness, here's a box plot of that versus some past scenarios:

In [ ]:
scenarios3 = {
    "default": (0.99, 0.75, 0.25),
    "low_b_and_c": (0.99, 0.6, 0.10),
    "high_b_and_c": (0.99, 0.9, 0.33),
    "final_proposal": (0.99, 0.80, 0.30)
}
box_sidebyside(scenarios3)